In [7]:
import pandas as pd
import numpy as np
import itertools

In [4]:
data = pd.read_csv('data/round.csv')

In [5]:
data.shape

(231125, 12)

In [6]:
data.head()

,Tournament_Year,Tournament_#,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Tee_Time,Round_Score,End_of_Event_Pos._(text),Finishing_Pos,Finishing_Pct
0,2003,10,16,656,1320,"Estes, Bob",1,11:50,66,T6,7.5,0.197917
1,2003,10,16,656,1361,"Forsman, Dan",1,11:00,69,T21,22.0,0.600694
2,2003,10,16,656,1761,"Mattiace, Len",1,13:00,75,24,24.0,0.656250
3,2003,10,16,656,1797,"Mediate, Rocco",1,12:10,72,T2,2.5,0.059028
4,2003,10,16,656,1920,"Parry, Craig",1,11:20,71,T25,26.5,0.725694


In [15]:
rounds = data.groupby(['Tournament_Year','Tournament_#','Round_Number'])

In [59]:
len(rounds)

2263

In [22]:
def label_players(player_inds,players):
    n = len(player_inds)
    for player in players:
        if player not in player_inds:
            player_inds[player] = n
            n += 1
    return player_inds

In [48]:
from operator import itemgetter
import matplotlib.pyplot as plt
%matplotlib inline

In [67]:
def fullprint(*args, **kwargs):
  from pprint import pprint
  opt = np.get_printoptions()
  np.set_printoptions(threshold='nan')
  pprint(*args, **kwargs)
  np.set_printoptions(**opt)

In [82]:
player_inds = {}
for u,df in enumerate(rounds):
    n = len(player_inds)
    if u==0:
        A = np.matrix(np.zeros((n,n)))
        G = np.matrix(np.zeros((n,n)))
    df = df[1]
    if u>75: continue
    player_inds = label_players(player_inds,df.Player_Number.values)
    if len(player_inds) > A.shape[0]:
        to_add = len(player_inds) - A.shape[0]
        A = np.vstack((A,np.zeros((to_add,A.shape[0]))))
        A = np.hstack((A,np.zeros((A.shape[0],to_add))))
        G = np.vstack((G,np.zeros((to_add,G.shape[0]))))
        G = np.hstack((G,np.zeros((G.shape[0],to_add))))
    df = df.sort_values('Round_Score')
    df.insert(5,'Index_Number',list(itemgetter(*df.Player_Number.values.tolist())(player_inds)))
    n = len(df)
    for row_ind in xrange(n):
        g_id = df.iloc[row_ind].Index_Number
        arr = np.zeros(n)
        arr[range(row_ind+1,n)] = 1
        A[g_id,df.Index_Number.values] += arr
        G[g_id,df.Index_Number.values] += np.ones(n)
    #fig, ax = plt.subplots(figsize=(20, 10))
    V = np.divide(A,G)
    V = np.nan_to_num(V)
    #plt.imshow(V)
    #plt.show()
    fullprint(V[0:5,0:5])

matrix([[ 0.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  1.,  1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  1.,  0.]])
matrix([[ 0. ,  1. ,  1. ,  1. ,  1. ],
        [ 0. ,  0. ,  0.5,  0.5,  0.5],
        [ 0. ,  0.5,  0. ,  0.5,  0.5],
        [ 0. ,  0.5,  0.5,  0. ,  0.5],
        [ 0. ,  0.5,  0.5,  0.5,  0. ]])
matrix([[ 0.        ,  0.66666667,  0.66666667,  0.66666667,  1.        ],
        [ 0.33333333,  0.        ,  0.33333333,  0.33333333,  0.66666667],
        [ 0.33333333,  0.66666667,  0.        ,  0.33333333,  0.66666667],
        [ 0.33333333,  0.66666667,  0.66666667,  0.        ,  0.66666667],
        [ 0.        ,  0.33333333,  0.33333333,  0.33333333,  0.        ]])
matrix([[ 0.  ,  0.75,  0.5 ,  0.5 ,  1.  ],
        [ 0.25,  0.  ,  0.25,  0.25,  0.75],
        [ 0.5 ,  0.75,  0.  ,  0.25,  0.75],
        [ 0.5 ,  0.75,  0.75,  0.  ,  0.75],
        [ 0.  ,  0.25,  0.25,  0.25,  0.  ]])
matrix([[ 0.  ,  0.75

In [ ]:
C = np.sum((G!=0),axis=0)
player_inds_reverse = dict((v, k) for k, v in player_inds.iteritems())
p_nums = [player_inds_reverse[ind] for ind in np.divide(np.sum(V,axis=0),C).argsort().tolist()[0][:20]]
for p in p_nums:
    print data[data['Player_Number']==p].iloc[0].Player_Name